In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

머신러닝의 학습 방법은 크게 지도 학습(supervised learning)과 비지도 학습(unsupervised learning)으로 나눌 수 있다.  
지도 학습이란 라벨링이 된 데이터를 학습시키는 것을 의미하며, 비지도 학습은 라벨링이 되지 않은 데이터를 학습시키는 것이다. 이때 라벨링이란 트레이닝 데이터에 정답이 표시된 것을 의미하며, 정답 부분을 타겟(클래스, 레이블) 데이터라고 한다.

타겟의 형태에 따라 지도 학습은 세부적으로 두 가지 종류로 나눌 수 있다. 타겟이 범주형인 경우에는 분류(classification) 문제라고 하고, 연속형 숫자인 경우에는 회귀(regression) 문제라고 한다. 타겟 데이터의 형태를 파악하고 풀려는 문제가 분류 문제인지 회귀 문제인지 파악하는 것이 중요하다.

k-최근접 이웃(kNN, k-Nearest Neighbor) 알고리즘  
k-최근접 이웃 알고리즘은 이해하기 쉽고, 자주 사용되는 방법으로 비교 대상이 되는 데이터 포인트 주변에 가장 가까이 존재하는 k개의 데이터와 비교해 가장 가까운 데이터 종류로 판별한다.

k-최근접 이웃 알고리즘은 학습 과정에서 게으른 학습(lazy learning) 방법을 사용한다. 게으른 학습은 트레이닝 데이터 전체를 메모리상에 보관하면서 테스트 데이터가 새로 들어왔을 때 바로 학습하는 것을 의미한다. 트레이닝 데이터를 메모리에 보관하므로 추가적인 학습시간 없이, 곧바로 학습 결과를 얻을 수 있다는 장점이 있지만 데이터가 지나치게 커서 메모리에 보관할 수 없을 경우에는 사용할 수 없다는 단점이 있다.

게으른 학습의 반대말은 열정적 학습(eager learning)이라 하는데, 트레이닝 데이터로 일정 기간 학습시킨 후 학습시킨 모형을 기반으로 테스트에 적용하는 방법으로 트레이닝 데이터는 학습 시에만 메모리에 보관되며 학습 이후에 테스트 데이터를 분류, 예측할 때 메모리에 보관할 필요가 없다.

게으른 학습과 열적적 학습의 차이는 학습 시간의 필요 유무에 따라 나뉘고 열정적 학습은 게으른 학습보다 메모리를 효율적으로 사용할 수 다는 장점이 있지만 학습 시간이 오래 걸린다는 단점이 있다.

붓꽃 데이터를 사용해서 붓꽃 종류를 구분하는 모델을 생성하고 학습시킨다.

In [8]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_iris = datasets.load_iris() # 붓꽃 데이터를 불러온다.
# from sklearn.datasets import load_iris
# raw_iris = load_iris()
# from sklearn.datasets import *
# raw_iris = load_iris()
# raw_iris

In [13]:
# 피쳐, 타겟 데이터 저장
X = raw_iris.data # 피쳐 데이터를 저장한다.
y = raw_iris.target # 타겟 데이터를 저장한다.
X.shape

(150, 4)

In [84]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
# train_test_split() 메소드로 피쳐, 타겟 데이터를 넘겨 트레이닝 데이터와 테스트 데이터로 나눈다.
# train_size 속성으로 트레이닝 데이터로 사용할 데이터의 비율을 지정한다.
# test_size 속성으로 테스트 데이터로 사용할 데이터의 비율을 지정한다.
# train_size, test_size 속성을 생략하면 트레이닝 데이터와 테스트 데이터를 75%와 25%로 분할한다.
# random_state 속성을 지정하면 매번 실행할 때 마다 같은 데이터를 얻어오게 할 수 있다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0) # 트레이닝 데이터와 테스트 데이터로 분할한다.
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape)

(112, 4) (112,) (38, 4) (38,)


In [80]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
# std_scale.fit(X_tn) # 적합
# X_tn_std = std_scale.transform(X_tn) # 트레이닝 데이터를 표준화 한다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [81]:
# 모델 생성 후 데이터 학습
from sklearn.neighbors import KNeighborsClassifier # k 최근접 이웃 알고리즘을 사용하기 위해 import 한다.
clf_knn = KNeighborsClassifier(n_neighbors=2) # k 최근접 이웃 알고리즘 객체(모델)를 만든다.
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 k 최근접 이웃 알고리즘을 학습시킨다.
clf_knn.fit(X_tn_std, y_tn)

KNeighborsClassifier(n_neighbors=2)

In [91]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
knn_pred = clf_knn.predict(X_te_std)
print(knn_pred)

[2 1 0 2 0 2 0 1 1 1 1 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


In [92]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(knn_pred)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, knn_pred)
print(accuracy)

0.9473684210526315


In [93]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(knn_pred)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, knn_pred)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  1  8]]


In [94]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(knn_pred)를 넘겨 분류 리포트를 만든다.
target_names = raw_iris.target_names
class_report = classification_report(y_te, knn_pred, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       0.94      0.94      0.94        16
   virginica       0.89      0.89      0.89         9

    accuracy                           0.95        38
   macro avg       0.94      0.94      0.94        38
weighted avg       0.95      0.95      0.95        38



In [97]:
for i in range(20):
    print(X_tn[i], X_tn_std[i], y_tn[i])

[5.9 3.  4.2 1.5] [ 0.01543995 -0.11925475  0.22512685  0.35579762] 1
[5.8 2.6 4.  1.2] [-0.09984503 -1.04039491  0.11355956 -0.02984109] 1
[6.8 3.  5.5 2.1] [ 1.05300481 -0.11925475  0.95031423  1.12707506] 2
[4.7 3.2 1.3 0.2] [-1.36797986  0.34131533 -1.39259884 -1.31530348] 0
[6.9 3.1 5.1 2.3] [1.1682898  0.11103029 0.72717965 1.38416753] 2
[5.  3.5 1.6 0.6] [-1.0221249   1.03217045 -1.2252479  -0.80111852] 0
[5.4 3.7 1.5 0.2] [-0.56098497  1.49274053 -1.28103155 -1.31530348] 0
[5.  2.  3.5 1. ] [-1.0221249  -2.42210516 -0.16535866 -0.28693357] 1
[6.5 3.  5.5 1.8] [ 0.70714986 -0.11925475  0.95031423  0.74143634] 2
[6.7 3.3 5.7 2.5] [0.93771983 0.57160037 1.06188152 1.64126001] 2
[6.  2.2 5.  1.5] [ 0.13072494 -1.96153508  0.671396    0.35579762] 2
[6.7 2.5 5.8 1.8] [ 0.93771983 -1.27067995  1.11766516  0.74143634] 2
[5.6 2.5 3.9 1.1] [-0.330415   -1.27067995  0.05777592 -0.15838733] 1
[7.7 3.  6.1 2.3] [ 2.09056967 -0.11925475  1.28501609  1.38416753] 2
[6.3 3.3 4.7 1.6] [0.4765798

In [100]:
newData = []
for i in range(20):
    new = std_scale.transform([X_tn[i]])
    newData.append(new)
for i in range(20):
    print(newData[i])

[[ 0.01543995 -0.11925475  0.22512685  0.35579762]]
[[-0.09984503 -1.04039491  0.11355956 -0.02984109]]
[[ 1.05300481 -0.11925475  0.95031423  1.12707506]]
[[-1.36797986  0.34131533 -1.39259884 -1.31530348]]
[[1.1682898  0.11103029 0.72717965 1.38416753]]
[[-1.0221249   1.03217045 -1.2252479  -0.80111852]]
[[-0.56098497  1.49274053 -1.28103155 -1.31530348]]
[[-1.0221249  -2.42210516 -0.16535866 -0.28693357]]
[[ 0.70714986 -0.11925475  0.95031423  0.74143634]]
[[0.93771983 0.57160037 1.06188152 1.64126001]]
[[ 0.13072494 -1.96153508  0.671396    0.35579762]]
[[ 0.93771983 -1.27067995  1.11766516  0.74143634]]
[[-0.330415   -1.27067995  0.05777592 -0.15838733]]
[[ 2.09056967 -0.11925475  1.28501609  1.38416753]]
[[0.47657989 0.57160037 0.50404507 0.48434386]]
[[-0.44569998 -1.50096499  0.00199227 -0.15838733]]
[[ 0.47657989 -0.81010987  0.61561236  0.74143634]]
[[ 0.47657989 -0.57982483  0.72717965  0.35579762]]
[[-1.13740989 -1.27067995  0.39247778  0.6128901 ]]
[[ 0.47657989 -1.2706799

In [101]:
for i in range(20):
    knn_pred = clf_knn.predict(newData[i])
    print(y_tn[i], knn_pred)

1 [1]
1 [1]
2 [2]
0 [0]
2 [2]
0 [0]
0 [0]
1 [1]
2 [2]
2 [2]
2 [1]
2 [2]
1 [1]
2 [2]
1 [1]
1 [1]
2 [2]
2 [2]
2 [1]
2 [2]
